# Homework 9-2

- Work an absorption-only problem
    - 5 mean free paths thick slab
    - Isotropic flux on left boundary (for incoming directions) with current of 1
        - Use Step, Diamond Difference, and Weighted with $\alpha=0.8$
        - Use $S_4$, $S_8$, $S_{12}$
        - Use as fine a spatial discretization as required for convergence
    - Analytic outgoing current on the right boundary is $2 E_3(5) = 1.755e-3$ (see App A)
    - Check accuracy vs. spatial discretization, auxiliary equation used, angular quadrature
- Extra credit: What would be the optimum value of $\alpha_n$

In [3]:
from scipy.special import legendre
from scipy.integrate import trapz
import numpy as np

In [4]:
num_sections = 10
t = 50
sigma = 5 / t
dx = t / num_sections
nang = 4

Auxiliary equations: these describe the average flux in terms of the flux at the edges

Step:

$$ 
\psi_{n,i+\frac{1}{2}} = \frac{\bar{S}_{n,i} + \frac{\mu_n}{\delta x_i} \psi_{n, i-\frac{1}{2}}}{\frac{\mu_n}{\delta x_i} + \sigma_{ti}}
$$
$$
\phi_{i \ell} = \sum_{n=1}^N w_n \bar{\psi}_{n,i} P_\ell (\mu_n)
$$

In [5]:
def step(psi_minus, mu=None):
    return psi_minus

Diamond Difference:

$$
\psi_{n,i+\frac{1}{2}} = 
\frac{
  \bar{S}_{n,i} 
      + 
  \left( 
    \frac{
      \mu_n
    }{
      \delta x_i
    } 
      - 
    \frac{
      \sigma_{ti}
    }{
      2
    } 
  \right) \psi_{n, i-\frac{1}{2}}
}
{
  \frac{
    \mu_n
  }{
    \delta x_i
  } 
    + 
  \frac{
    \sigma_{ti}
  }{
    2
  }
}
$$

In [6]:
def diamond_difference(psi_minus, mu):
    psi_plus = s + (mu / dx - sigma / 2) * psi_minus
    psi_plus /= mu / dx + sigma / 2
    return psi_plus

Weighted diamond difference:

$$
\psi_{n,i+\frac{1}{2}} = 
\frac{\bar{S}_{n,i}+ \left(\frac{\mu_n}{\delta x_i}-(1 - \alpha) \sigma_{ti}\right) \psi_{n, i-\frac{1}{2}}}{\frac{\mu_n}{\delta x_i} + \alpha \sigma_{ti}}
$$

In [7]:
def weighted_diamond_difference(psi_minus, mu, alpha=0.8):
    psi_plus = s + (mu / dx - (1 - alpha) * sigma) * psi_minus
    psi_plus /= (mu / dx) + alpha * sigma
    return psi_plus

Since we know the incoming current on the left boundary, we will proceed from left to right

This is a one-group problem, so there are no need for outer iterations

In [36]:
def set_quadrature(nang):
    wt = []
    mu = []

    if(nang==2):
        wt.append(1.)

        mu.append(.5773502691)

    elif(nang==4):
        wt.append(.6521451549)
        wt.append(.3478548451)

        mu.append(.3399810435)
        mu.append(.8611363115)

    elif(nang==8):
        wt.append(.3626837834)
        wt.append(.3137066459)
        wt.append(.2223810344)
        wt.append(.1012285363)

        mu.append(.1834346424)
        mu.append(.5255324099)
        mu.append(.7966664774)
        mu.append(.9602898564)

    elif(nang==12):
        wt.append(0.2491470458)
        wt.append(0.2334925365)
        wt.append(0.2031674267)
        wt.append(0.1600783286)
        wt.append(0.1069393260)
        wt.append(0.0471753364)

        mu.append(0.1252334085)
        mu.append(0.3678314989)
        mu.append(0.5873179542)
        mu.append(0.7699026741)
        mu.append(0.9041172563)
        mu.append(0.9815606342)
    
    #mus = np.array([-m for m in mu] + mu)
    #weights = np.array([w / 2 for w in wt] + [w / 2 for w in wt])
    
    mus = np.array(mu)
    weights = np.array(wt)
    
    try:
        assert(np.allclose(np.sum(weights), np.array(1.0)))
    except AssertionError as ae:
        print(np.sum(weights))
        raise ae
    
    return mus, weights

In [38]:
mu, wt = set_quadrature(12)

In [41]:
np.concatenate([mu, -mu])

array([ 0.12523341,  0.3678315 ,  0.58731795,  0.76990267,  0.90411726,
        0.98156063, -0.12523341, -0.3678315 , -0.58731795, -0.76990267,
       -0.90411726, -0.98156063])

Set initial convergence and error values

In [28]:
def s(mu, flux):
    tot = 0
    for l in range(len(wt)):
        m = mu[l]
        w = wt[l] / 2
        tot += (2 * l + 1) * (w * legendre(l)(m) + w * legendre(l)(-m))
    return tot

In [29]:
s(0.3, mu)

0.49105917922126346

In [ ]:
flux_func = step
loop_counter = 0
eps = 0.00000001
conv = 100000.

flux = np.zeros(num_sections)
left = np.zeros(num_sections)
right = np.zeros(num_sections)

while abs(conv) > eps:
    loop_counter += 1
    # Loop over directions
    for a in range(nang):
        phi = 0.0
        muabs = abs(mu[a])
        for ix in range(num_sections):
            phi_left = phi_right
            phi_right = flux_func()
            flux_avg = 
            flux[ix] += wt[a] * flux_avg